<a href="https://colab.research.google.com/github/ajagota7/Reward-Shaping/blob/main/Lifegate_straight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import numpy as np
import os
from google.colab import drive
import pickle
# np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
from scipy.optimize import minimize
import random
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import torch
import sys
import plotly.graph_objects as go

# deadend dependencies

In [2]:
!git clone https://github.com/microsoft/med-deadend.git


Cloning into 'med-deadend'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 130 (delta 52), reused 77 (delta 22), pack-reused 0
Receiving objects: 100% (130/130), 395.48 KiB | 10.14 MiB/s, done.
Resolving deltas: 100% (52/52), done.


# shaping dependencies

In [3]:
!git clone https://github.com/ajagota7/Shaping.git

Cloning into 'Shaping'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 101 (delta 49), reused 74 (delta 28), pack-reused 0
Receiving objects: 100% (101/101), 11.74 MiB | 23.21 MiB/s, done.
Resolving deltas: 100% (49/49), done.


In [4]:
# %cd /content/Shaping

In [5]:
# !git pull origin main

In [6]:
# cd /content/

In [7]:
# %cd /content/Shaping

import zipfile

with zipfile.ZipFile('/content/Shaping/lifegate_1.zip', 'r') as zip_ref:
    # zip_ref.extractall('/content/med-deadend/lifegate/results/lifegate_1')
    zip_ref.extractall('/content/Shaping/')

In [8]:
import sys
sys.path.append('/content/med-deadend/lifegate')
sys.path.append('/content/Shaping/')



In [9]:
import q_networks

In [10]:
# %cd /content/med-deadend/lifegate


# results_dir = 'results/lifegate_1/'
results_dir = '/content/Shaping/'
# Load the Q tables from the primary learning agent, Q_D and Q_R value functions
with open(results_dir+'tabular_qnet.pkl', 'rb') as fq:
    ai = pickle.load(fq)

with open(results_dir+'tabular_qd.pkl', 'rb') as fd:
    ai_d = pickle.load(fd)

with open(results_dir+'tabular_qr.pkl', 'rb') as fr:
    ai_r = pickle.load(fr)

In [11]:
q_table = np.zeros((10, 10, 5))
q_d = np.zeros_like(q_table)
q_r = np.zeros_like(q_table)


for i in range(10):
    for j in range(10):
        for a in range(5):
            key = tuple([j, i, a])
            try:
                q_table[i,j,a] = ai.q[key]
                q_d[i,j,a] = ai_d.q[key]
                q_r[i,j,a] = ai_r.q[key]
            except:
                pass

In [12]:
import yaml
import random
from lifegate import LifeGate
params = yaml.safe_load(open(results_dir+'config.yaml', 'r'))
np.random.seed(seed=params['random_seed'])
random.seed(params['random_seed'])
random_state = np.random.RandomState(params['random_seed'])

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [13]:
env = LifeGate(max_steps=params['episode_max_len'], state_mode='tabular',
                        rendering=True, image_saving=False, render_dir=None, rng=random_state, death_drag = 0.0)

In [14]:
env_30 = LifeGate(max_steps=30, state_mode='tabular',
                        rendering=True, image_saving=False, render_dir=None, rng=random_state, death_drag = 0.1)

In [15]:
import Shaping
from Shaping import *
# %cd /content/Shaping

from choose_actions import action_probs_top_n_epsilon
from shaping_features import *
from gen_policies import *
from IS import *
from subset_policies import *
from v_pi_e import *
from optimization import *
from neural_net import *
from prep_variance import *
from SCOPE_variance import SCOPE_variance

# SCOPE straight

In [140]:
class SCOPE_straight(object):

  def __init__(self, model, gamma, num_bootstraps, pi_b, P_pi_b, P_pi_e, percent_to_estimate_phi, dtype):
        self.model = model
        self.gamma = gamma
        self.num_bootstraps = num_bootstraps
        self.pi_b = pi_b
        self.P_pi_b = P_pi_b
        self.P_pi_e = P_pi_e
        self.dtype = dtype

        self.percent_to_estimate_phi = percent_to_estimate_phi
        # self.num_epochs = num_epochs

  def subset_policies(self):
    # seed_value = 0
    # np.random.seed(seed_value)
    num_policies = len(self.pi_b)
    num_policies_to_estimate_phi = int(num_policies * self.percent_to_estimate_phi)

    policies_for_scope = self.pi_b[num_policies_to_estimate_phi:]
    policies_for_phi = self.pi_b[:num_policies_to_estimate_phi]

    return policies_for_phi, policies_for_scope

  def prep_policies(self, chosen_policies):
      # Initialize lists to store axis data for each policy
      timesteps = []
      # states = []
      # state_first = []
      # state_last = []
      actions = []
      rewards = []
      # gamma_last = []
      # weight_last = []
      # weight_first = []
      # all_weights_temp, weights = calculate_importance_weights(P_pi_e, P_pi_b, pi_b)
      weights = calculate_importance_weights(self.P_pi_e, self.P_pi_b, chosen_policies)
      psi = []

      states_current = []
      states_next = []

      for index, policy in enumerate(chosen_policies):
          policy_array = np.array(policy)

          timesteps.append(policy_array['timestep'].astype(int))
          actions.append(policy_array['action'])
          rewards.append(policy_array['reward'].astype(float))

          states_next.append(policy_array['state_next'])
          states_current.append(policy_array['state'])

      return timesteps, rewards, states_next, states_current, weights, actions

  def padding_IS_terms(self, timesteps, actions, rewards, weights):

    # Find the maximum length among all lists
    max_length = max(len(traj) for traj in timesteps)

    # Define the padding values
    zero_padding = 0

    # Pad each list to match the maximum length
    padded_timesteps = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in timesteps]
    padded_rewards = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in rewards]
    padded_actions = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in actions]
    padded_weights = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in weights]

    return padded_timesteps, padded_rewards, padded_actions, padded_weights


  def tensorize_IS_terms(self, padded_timesteps, padded_rewards, padded_weights):

    padded_timestep_tensors = torch.tensor(padded_timesteps, dtype = self.dtype)
    padded_reward_tensors = torch.tensor(padded_rewards, dtype = self.dtype)
    padded_weight_tensors = torch.tensor(padded_weights, dtype = self.dtype)

    return padded_timestep_tensors, padded_reward_tensors, padded_weight_tensors

  def padding_states(self, states_next, states_current):
    # Find the maximum length of trajectories
    max_length = max(len(trajectory) for trajectory in states_current)

    zero_padding = 0

    # Pad each trajectory to make them all the same length
    padded_states_next = [
        [list(item) for item in trajectory] + [[0, 0]] * (max_length - len(trajectory))
        for trajectory in states_next
    ]

    # Pad each trajectory to make them all the same length
    padded_states_current = [
        [list(item) for item in trajectory] + [[0, 0]] * (max_length - len(trajectory))
        for trajectory in states_current
    ]

    return padded_states_next, padded_states_current


  def tensorize_padded_terms(self, padded_states_next, padded_states_current):
    padded_states_next_tensors = torch.tensor(padded_states_next, dtype = self.dtype)
    padded_states_current_tensors = torch.tensor(padded_states_current, dtype = self.dtype)

    return padded_states_next_tensors, padded_states_current_tensors


  def prepare_IS(self):
    timesteps, rewards, states_next, states_current, weights, actions = self.prep_policies(self.pi_b)
    padded_timesteps, padded_rewards, padded_actions, padded_weights = self.padding_IS_terms(timesteps, actions, rewards, weights)
    padded_timestep_tensors, padded_reward_tensors, padded_weight_tensors = self.tensorize_IS_terms(padded_timesteps, padded_rewards, padded_weights)

    return padded_timestep_tensors, padded_reward_tensors, padded_weight_tensors

  def prepare_SCOPE(self, policies):
    timesteps, rewards, states_next, states_current, weights, actions = self.prep_policies(policies)
    padded_timesteps, padded_rewards, padded_actions, padded_weights = self.padding_IS_terms(timesteps, actions, rewards, weights)
    padded_timestep_tensors, padded_reward_tensors, padded_weight_tensors = self.tensorize_IS_terms(padded_timesteps, padded_rewards, padded_weights)
    padded_states_next, padded_states_current = self.padding_states(states_next, states_current)
    padded_states_next_tensors, padded_states_current_tensors = self.tensorize_padded_terms(padded_states_next, padded_states_current)

    return padded_timestep_tensors, padded_reward_tensors, padded_weight_tensors, padded_states_next_tensors, padded_states_current_tensors

  def prepare_SCOPE_phi(self):
    phi_set,_ = self.subset_policies()
    padded_timestep_tensors, padded_reward_tensors, padded_weight_tensors, padded_states_next_tensors, padded_states_current_tensors = self.prepare_SCOPE(phi_set)

    return padded_timestep_tensors, padded_reward_tensors, padded_weight_tensors, padded_states_next_tensors, padded_states_current_tensors

  def prepare_SCOPE_test(self):
    _, scope_set = self.subset_policies()
    padded_timestep_tensors, padded_reward_tensors, padded_weight_tensors, padded_states_next_tensors, padded_states_current_tensors = self.prepare_SCOPE(scope_set)

    return padded_timestep_tensors, padded_reward_tensors, padded_weight_tensors, padded_states_next_tensors, padded_states_current_tensors

  def pass_states(self, model, padded_states_next_tensors, padded_states_current_tensors):
    states_next_output = self.model(padded_states_next_tensors)
    states_current_output = self.model(padded_states_current_tensors)

    return states_next_output.squeeze(), states_current_output.squeeze()

  def bootstrap_straight(self, padded_timestep_tensors, padded_reward_tensors, padded_weight_tensors, states_next_output, states_current_output):
    seed = 42
    torch.manual_seed(seed)

    num_samples = self.num_bootstraps
    num_bootstraps_lin = num_samples*padded_timestep_tensors.shape[0]

    # Sample indices with replacement
    sampled_indices = torch.randint(0, len(padded_timestep_tensors), size=(num_bootstraps_lin,), dtype=torch.long)

    reshaped_size = (num_samples, padded_timestep_tensors.shape[0], padded_timestep_tensors.shape[1])

    timestep_bootstraps = padded_timestep_tensors[sampled_indices].view(reshaped_size)
    rewards_bootstraps = padded_reward_tensors[sampled_indices].view(reshaped_size)
    weights_bootstraps = padded_weight_tensors[sampled_indices].view(reshaped_size)

    phi_states_next_bootstraps = states_next_output[sampled_indices].view(reshaped_size)
    phi_states_current_bootstraps = states_current_output[sampled_indices].view(reshaped_size)

    return timestep_bootstraps, rewards_bootstraps, weights_bootstraps, phi_states_next_bootstraps, phi_states_current_bootstraps

  def bootstrap_IS(self, padded_timestep_tensors, padded_reward_tensors, padded_weight_tensors):
    seed = 42
    torch.manual_seed(seed)

    num_samples = self.num_bootstraps
    num_bootstraps_lin = num_samples*padded_timestep_tensors.shape[0]

    # Sample indices with replacement
    sampled_indices = torch.randint(0, len(padded_timestep_tensors), size=(num_bootstraps_lin,), dtype=torch.long)

    reshaped_size = (num_samples, padded_timestep_tensors.shape[0], padded_timestep_tensors.shape[1])

    timestep_bootstraps = padded_timestep_tensors[sampled_indices].view(reshaped_size)
    rewards_bootstraps = padded_reward_tensors[sampled_indices].view(reshaped_size)
    weights_bootstraps = padded_weight_tensors[sampled_indices].view(reshaped_size)
    return timestep_bootstraps, rewards_bootstraps, weights_bootstraps


  def pass_then_boostraps(self, padded_states_next_tensors, padded_states_current_tensors, padded_timestep_tensors, padded_reward_tensors, padded_weight_tensors):
    states_next_output, states_current_output = self.pass_states(self.model, padded_states_next_tensors, padded_states_current_tensors)
    timestep_bootstraps, rewards_bootstraps, weights_bootstraps, phi_states_next_bootstraps, phi_states_current_bootstraps = self.bootstrap_straight(padded_timestep_tensors, padded_reward_tensors, padded_weight_tensors, states_next_output, states_current_output)

    return timestep_bootstraps, rewards_bootstraps, weights_bootstraps, phi_states_next_bootstraps, phi_states_current_bootstraps

  def calc_variance_IS(self, timestep_bootstraps, weights_bootstraps, rewards_bootstraps):
    IS_bootstraps = self.gamma**(timestep_bootstraps)* weights_bootstraps *(rewards_bootstraps)

    # Step 1: Sum along the third dimension
    sum_IS_trajectories = torch.sum(IS_bootstraps, dim=2)  # Shape: [1000, 1000]

    # Step 2: Take the mean along the second dimension
    mean_IS_sum = torch.mean(sum_IS_trajectories, dim=1)  # Shape: [1000]

    # Step 3: Calculate the variance across the first dimension
    IS_variance = torch.var(mean_IS_sum)  # A single scalar value

    IS_mean = torch.mean(mean_IS_sum) # A single scalar value

    return IS_mean, IS_variance

  def IS_pipeline(self):
    padded_timestep_tensors_IS, padded_reward_tensors_IS, padded_weight_tensors_IS = self.prepare_IS()
    timestep_bootstraps_IS, rewards_bootstraps_IS, weights_bootstraps_IS = self.bootstrap_IS(padded_timestep_tensors_IS, padded_reward_tensors_IS, padded_weight_tensors_IS)
    IS_mean, IS_variance = self.calc_variance_IS(timestep_bootstraps_IS, rewards_bootstraps_IS, weights_bootstraps_IS)

    return IS_mean, IS_variance


  def calc_variance_straight(self, timestep_bootstraps, weights_bootstraps, rewards_bootstraps, phi_states_next_bootstraps, phi_states_current_bootstraps):

    # IS_boostraps = self.gamma**(timestep_bootstraps)* weights_bootstraps *(rewards_bootstraps)
    scope_bootstraps = self.gamma**(timestep_bootstraps)* weights_bootstraps *(rewards_bootstraps  +self.gamma*phi_states_next_bootstraps - phi_states_current_bootstraps)

    # # Step 1: Sum along the third dimension
    # sum_IS_trajectories = torch.sum(IS_boostraps, dim=2)  # Shape: [1000, 1000]

    # # Step 2: Take the mean along the second dimension
    # mean_IS_sum = torch.mean(sum_IS_trajectories, dim=1)  # Shape: [1000]

    # # Step 3: Calculate the variance across the first dimension
    # IS_variance = torch.var(mean_IS_sum)  # A single scalar value


    # Step 1: Sum along the third dimension
    sum_scope_trajectories = torch.sum(scope_bootstraps, dim=2)  # Shape: [1000, 1000]

    # Step 2: Take the mean along the second dimension
    mean_scope_sum = torch.mean(sum_scope_trajectories, dim=1)  # Shape: [1000]

    # Step 3: Calculate the variance across the first dimension
    scope_variance = torch.var(mean_scope_sum)  # A single scalar value

    scope_mean = torch.mean(mean_scope_sum) # A single scalar value

    return scope_mean, scope_variance

  def train_var_scope(self, num_epochs, learning_rate):


      padded_timestep_tensors_IS, padded_reward_tensors_IS, padded_weight_tensors_IS = self.prepare_IS()
      timestep_bootstraps_IS, rewards_bootstraps_IS, weights_bootstraps_IS = self.bootstrap_IS(padded_timestep_tensors_IS, padded_reward_tensors_IS, padded_weight_tensors_IS)
      IS_mean, IS_variance = self.calc_variance_IS(timestep_bootstraps_IS, rewards_bootstraps_IS, weights_bootstraps_IS)

      padded_timestep_tensors, padded_reward_tensors, padded_weight_tensors, padded_states_next_tensors, padded_states_current_tensors = self.prepare_SCOPE_phi()


      self.model.train()

      # Enable anomaly detection
      torch.autograd.set_detect_anomaly(True)

      optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)

      for epoch in range(num_epochs):
          total_loss = 0

          timestep_bootstraps, rewards_bootstraps, weights_bootstraps, phi_states_next_bootstraps, phi_states_current_bootstraps = self.pass_then_boostraps(padded_states_next_tensors, padded_states_current_tensors, padded_timestep_tensors, padded_reward_tensors, padded_weight_tensors)
          SCOPE_mean, SCOPE_variance = self.calc_variance_straight(timestep_bootstraps, weights_bootstraps, rewards_bootstraps, phi_states_next_bootstraps, phi_states_current_bootstraps)
          print(f"Epoch {epoch+1}")
          print("IS variance: ", IS_variance)
          print("SCOPE Var loss: ", SCOPE_variance)
          # print("MSE loss: ", mse_loss.item())


          tot = SCOPE_variance
          # tot = variance_loss + mse_loss

          optimizer.zero_grad()

          # Retain the graph to avoid clearing it before backward pass
          tot.backward(retain_graph=True)

          optimizer.step()

          total_loss += tot.item()

          print(f"Total Loss: {total_loss}")
          print("-" * 40)

      # Disable anomaly detection after running the code
      torch.autograd.set_detect_anomaly(False)

      for name, param in self.model.named_parameters():
          if param.requires_grad:
              print(f"Parameter name: {name}")
              print(f"Weights: {param.data}")

      return self.model

  def evaluate_scope(self):
    self.model.eval()
    padded_timestep_tensors, padded_reward_tensors, padded_weight_tensors, padded_states_next_tensors, padded_states_current_tensors = self.prepare_SCOPE_test()
    timestep_bootstraps, rewards_bootstraps, weights_bootstraps, phi_states_next_bootstraps, phi_states_current_bootstraps = self.pass_then_boostraps(padded_states_next_tensors, padded_states_current_tensors, padded_timestep_tensors, padded_reward_tensors, padded_weight_tensors)
    SCOPE_mean, SCOPE_variance = self.calc_variance_straight(timestep_bootstraps, weights_bootstraps, rewards_bootstraps, phi_states_next_bootstraps, phi_states_current_bootstraps)

    return SCOPE_mean, SCOPE_variance


  # -----------------------
  # Heatmaps for lifegate
  # -----------------------
  def get_model_output_dict(self):

    self.model.eval()

    # Initialize an empty dictionary to store data
    data = {}

    # Loop through all combinations from [0,0] to [9,9]
    for i in range(10):
      for j in range(10):
          # Prepare input data
          input_data = torch.tensor([i, j], dtype=torch.float64)

          # Pass input through the self.model
          output = self.model(input_data)

          # Store data in the dictionary
          data[(i, j)] = output.item()

    return data

  def plot_heatmap(self, data):
    values = np.zeros((10, 10))
    for (x, y), value in data.items():
        values[y, x] = value

    # Create the heatmap
    fig = go.Figure(data=go.Heatmap(z=values, colorscale='viridis'))

    # Add colorbar
    fig.update_layout(coloraxis_colorbar=dict(title='Values',
                                              ticks='outside',
                                              tickvals=[np.min(values), np.max(values)],
                                              ticktext=[np.min(values), np.max(values)]))

    # Add labels and title
    fig.update_layout(xaxis=dict(tickvals=np.arange(10), ticktext=list(range(10)), title='X'),
                      yaxis=dict(tickvals=np.arange(9, -1, -1), ticktext=list(range(9, -1, -1)), title='Y', autorange="reversed"),
                      title='Heatmap')

    fig.show()

  def get_heatmap(self):
    data = self.get_model_output_dict()
    self.plot_heatmap(data)


# Test class

In [98]:
P_pi_b = action_probs_top_n_epsilon(q_table, 1, 0.4)
pi_b = experiment_actions(200, env_30, P_pi_b)
P_pi_e = action_probs_top_n_epsilon(q_table, 2, 0.05)
# pi_e = experiment_actions(200, env, P_pi_e)
model_200_0p99 = CustomizableFeatureNet(input_dim=2, hidden_dims=[16, 16], output_dim=1, dtype = torch.float64)
test_200_0p99 = SCOPE_straight(model_200_0p99, 0.99, 10000, pi_b, P_pi_b, P_pi_e, 0.3, dtype = torch.float64)


In [99]:
test_200_0p99.IS_pipeline()

(tensor(0.0176, dtype=torch.float64), tensor(0.0001, dtype=torch.float64))

In [109]:
pi_e = experiment_actions(1000, env_30, P_pi_e)


In [110]:
calc_V_pi_e(pi_e)

0.02696880170276661

In [112]:
test_200_0p99.evaluate_scope()

(tensor(0.0358, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(0.0003, dtype=torch.float64, grad_fn=<VarBackward0>))

In [111]:
test_200_0p99.train_var_scope(50, 0.001)

Epoch 1
IS variance:  tensor(0.0001, dtype=torch.float64)
SCOPE Var loss:  tensor(0.0003, dtype=torch.float64, grad_fn=<VarBackward0>)
Total Loss: 0.00028689004977398017
----------------------------------------
Epoch 2
IS variance:  tensor(0.0001, dtype=torch.float64)
SCOPE Var loss:  tensor(8.1809e-06, dtype=torch.float64, grad_fn=<VarBackward0>)
Total Loss: 8.180862088559544e-06
----------------------------------------
Epoch 3
IS variance:  tensor(0.0001, dtype=torch.float64)
SCOPE Var loss:  tensor(8.3946e-06, dtype=torch.float64, grad_fn=<VarBackward0>)
Total Loss: 8.394565779552828e-06
----------------------------------------
Epoch 4
IS variance:  tensor(0.0001, dtype=torch.float64)
SCOPE Var loss:  tensor(8.1937e-06, dtype=torch.float64, grad_fn=<VarBackward0>)
Total Loss: 8.193661968978786e-06
----------------------------------------
Epoch 5
IS variance:  tensor(0.0001, dtype=torch.float64)
SCOPE Var loss:  tensor(8.1998e-06, dtype=torch.float64, grad_fn=<VarBackward0>)
Total Lo

CustomizableFeatureNet(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=2, out_features=16, bias=True)
    (1): Linear(in_features=16, out_features=16, bias=True)
  )
  (output_layer): Linear(in_features=16, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

# Test random policy

In [141]:
P_pi_b = action_probs_top_n_epsilon(q_table, 1, 0.4)
pi_b = experiment_actions(200, env_30, P_pi_b)
P_pi_e = action_probs_top_n_epsilon(q_table, 5, 0.05)
# pi_e = experiment_actions(200, env_30, P_pi_e)
model_200_random = CustomizableFeatureNet(input_dim=2, hidden_dims=[16, 16], output_dim=1, dtype = torch.float64)
test_200_random = SCOPE_straight(model_200_random, 0.99, 10000, pi_b, P_pi_b, P_pi_e, 0.3, dtype = torch.float64)


In [159]:
test_200_random.train_var_scope(200, 0.001)

Epoch 1
IS variance:  tensor(8.9415e-05, dtype=torch.float64)
SCOPE Var loss:  tensor(0.0005, dtype=torch.float64, grad_fn=<VarBackward0>)
Total Loss: 0.0005218583187486271
----------------------------------------
Epoch 2
IS variance:  tensor(8.9415e-05, dtype=torch.float64)
SCOPE Var loss:  tensor(9.8199e-06, dtype=torch.float64, grad_fn=<VarBackward0>)
Total Loss: 9.819862798753365e-06
----------------------------------------
Epoch 3
IS variance:  tensor(8.9415e-05, dtype=torch.float64)
SCOPE Var loss:  tensor(1.0250e-05, dtype=torch.float64, grad_fn=<VarBackward0>)
Total Loss: 1.024958128773479e-05
----------------------------------------
Epoch 4
IS variance:  tensor(8.9415e-05, dtype=torch.float64)
SCOPE Var loss:  tensor(9.3286e-06, dtype=torch.float64, grad_fn=<VarBackward0>)
Total Loss: 9.328558541237725e-06
----------------------------------------
Epoch 5
IS variance:  tensor(8.9415e-05, dtype=torch.float64)
SCOPE Var loss:  tensor(8.3393e-06, dtype=torch.float64, grad_fn=<VarB

CustomizableFeatureNet(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=2, out_features=16, bias=True)
    (1): Linear(in_features=16, out_features=16, bias=True)
  )
  (output_layer): Linear(in_features=16, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [160]:
test_200_random.get_heatmap()

In [150]:
test_200_random.IS_pipeline()

(tensor(-0.0130, dtype=torch.float64), tensor(8.9415e-05, dtype=torch.float64))

In [151]:
pi_e = experiment_actions(1000, env_30, P_pi_e)


In [152]:
calc_V_pi_e(pi_e)

-0.12203056843017494

In [161]:
test_200_random.evaluate_scope()

(tensor(-0.0521, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(0.0003, dtype=torch.float64, grad_fn=<VarBackward0>))

# Test random pi_b

In [162]:
P_pi_b = action_probs_top_n_epsilon(q_table, 5, 0.4)
pi_b = experiment_actions(200, env_30, P_pi_b)
P_pi_e = action_probs_top_n_epsilon(q_table, 1, 0.05)
# pi_e = experiment_actions(200, env_30, P_pi_e)
model_200_random_pi_b = CustomizableFeatureNet(input_dim=2, hidden_dims=[16, 16], output_dim=1, dtype = torch.float64)
test_200_random_pi_b = SCOPE_straight(model_200_random_pi_b, 0.99, 10000, pi_b, P_pi_b, P_pi_e, 0.3, dtype = torch.float64)


In [169]:
test_200_random_pi_b.train_var_scope(200, 0.001)

Epoch 1
IS variance:  tensor(1.0768e-11, dtype=torch.float64)
SCOPE Var loss:  tensor(0.0287, dtype=torch.float64, grad_fn=<VarBackward0>)
Total Loss: 0.02867102506653055
----------------------------------------
Epoch 2
IS variance:  tensor(1.0768e-11, dtype=torch.float64)
SCOPE Var loss:  tensor(0.0005, dtype=torch.float64, grad_fn=<VarBackward0>)
Total Loss: 0.0005117103406371684
----------------------------------------
Epoch 3
IS variance:  tensor(1.0768e-11, dtype=torch.float64)
SCOPE Var loss:  tensor(0.0005, dtype=torch.float64, grad_fn=<VarBackward0>)
Total Loss: 0.0005069477039305052
----------------------------------------
Epoch 4
IS variance:  tensor(1.0768e-11, dtype=torch.float64)
SCOPE Var loss:  tensor(0.0005, dtype=torch.float64, grad_fn=<VarBackward0>)
Total Loss: 0.0005035621463707624
----------------------------------------
Epoch 5
IS variance:  tensor(1.0768e-11, dtype=torch.float64)
SCOPE Var loss:  tensor(0.0005, dtype=torch.float64, grad_fn=<VarBackward0>)
Total L

CustomizableFeatureNet(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=2, out_features=16, bias=True)
    (1): Linear(in_features=16, out_features=16, bias=True)
  )
  (output_layer): Linear(in_features=16, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [170]:
test_200_random_pi_b.get_heatmap()

In [165]:
test_200_random_pi_b.IS_pipeline()

(tensor(-3.4482e-06, dtype=torch.float64),
 tensor(1.0768e-11, dtype=torch.float64))

In [166]:
pi_e = experiment_actions(1000, env_30, P_pi_e)


In [167]:
calc_V_pi_e(pi_e)

0.15990763270320862

In [171]:
test_200_random_pi_b.evaluate_scope()

(tensor(0.0015, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(1.3966e-05, dtype=torch.float64, grad_fn=<VarBackward0>))

In [107]:
# Get the state_dict of the model
model_state_dict = test_200_0p99.model.state_dict()

# Print the keys to see the structure of the state_dict
print(model_state_dict.keys())

# Extract and print the weights of each layer
for name, param in model_state_dict.items():
    if 'weight' in name:
        print(f"Layer: {name}")
        print(param)

odict_keys(['hidden_layers.0.weight', 'hidden_layers.0.bias', 'hidden_layers.1.weight', 'hidden_layers.1.bias', 'output_layer.weight', 'output_layer.bias'])
Layer: hidden_layers.0.weight
tensor([[-0.5992, -0.0774],
        [ 0.4740, -0.2568],
        [ 0.3057,  0.1028],
        [ 0.1205, -0.2533],
        [ 0.6123,  0.1611],
        [-0.2607, -0.6321],
        [-0.7033, -0.3404],
        [-0.5834, -0.1342],
        [ 0.1821, -0.5556],
        [-0.4225,  0.3797],
        [-0.4621, -0.6912],
        [ 0.6232, -0.4203],
        [ 0.2262, -0.6769],
        [ 0.5225,  0.2460],
        [ 0.6153, -0.0401],
        [-0.1569, -0.2980]], dtype=torch.float64)
Layer: hidden_layers.1.weight
tensor([[ 1.0516e-01, -1.5570e-01, -8.9501e-02,  2.2105e-01,  2.2795e-01,
          6.3637e-02,  3.5722e-02, -1.2352e-01, -1.2532e-01, -1.9321e-01,
          1.7001e-01,  2.7690e-01,  5.0010e-02,  4.4123e-02, -1.7390e-01,
         -3.3544e-02],
        [ 2.9563e-02, -1.6539e-01,  1.8811e-01,  2.4416e-02, -2.0411

In [ ]:
scope_testing = SCOPE_straight(model, 0.9, 10000, pi_b, P_pi_b, P_pi_e, dtype = torch.float64)

In [ ]:
padded_timestep_tensors, padded_reward_tensors, padded_weight_tensors, padded_states_next_tensors, padded_states_current_tensors = scope_testing.prepare()
timestep_bootstraps, rewards_bootstraps, weights_bootstraps, phi_states_next_bootstraps, phi_states_current_bootstraps = scope_testing.pass_then_boostraps(model, padded_states_next_tensors, padded_states_current_tensors, padded_timestep_tensors, padded_reward_tensors, padded_weight_tensors)
IS_variance, scope_variance = scope_testing.calc_variance_straight(timestep_bootstraps, weights_bootstraps, rewards_bootstraps, phi_states_next_bootstraps, phi_states_current_bootstraps)

# train var scope

In [17]:
def train_var_scope(model, num_epochs, learning_rate, test1):

    padded_timestep_tensors, padded_reward_tensors, padded_weight_tensors, padded_states_next_tensors, padded_states_current_tensors = test1.prepare()

    model.train()

    # Enable anomaly detection
    torch.autograd.set_detect_anomaly(True)

    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        total_loss = 0

        # Forward pass
        # states_output, states_first_output, states_last_output = test1.pass_states(model, padded_state_tensors, states_first_tensor, states_last_tensor)
        # sums_states_weight_diff = test1.states_weight_diff_sums(states_output, padded_weight_diff_tensors)
        # gamma_weights_states_last_sub_states_first = test1.last_first_terms_operations(gamma_weights_last_tensor, states_last_output, states_first_output, weight_first_tensor)
        # # sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first, samples_all_shaping, samples_IS_SCOPE = test1.bootstrap_shaping_terms(sums_states_weight_diff, gamma_weights_states_last_sub_states_first, IS_tensor)

        # samples_IS, sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first, samples_all_shaping, samples_IS_SCOPE = test1.bootstrap_all_terms(sums_states_weight_diff, gamma_weights_states_last_sub_states_first, IS_tensor, padded_psi_tensors)


        # Calculate MSE loss between states_output and padded_state_tensors
        # mse_loss = F.mse_loss(states_output, padded_state_tensors)

        # E_IS_sq, E_IS_all_sq, E_s_wdiff_sq, E_s_wdiff_all_sq, E_IS_SCOPE, E_IS_E_SCOPE, _, variance_loss, E_IS, E_SCOPE = calculate_shaped_variance_play(samples_IS, sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first, samples_all_shaping, samples_IS_SCOPE)

        timestep_bootstraps, rewards_bootstraps, weights_bootstraps, phi_states_next_bootstraps, phi_states_current_bootstraps = test1.pass_then_boostraps(model, padded_states_next_tensors, padded_states_current_tensors, padded_timestep_tensors, padded_reward_tensors, padded_weight_tensors)
        IS_variance, variance_loss = test1.calc_variance_straight(timestep_bootstraps, weights_bootstraps, rewards_bootstraps, phi_states_next_bootstraps, phi_states_current_bootstraps)
        print(f"Epoch {epoch+1}")
        print("IS variance: ", IS_variance)
        print("SCOPE Var loss: ", variance_loss)
        # print("MSE loss: ", mse_loss.item())


        tot = variance_loss
        # tot = variance_loss + mse_loss

        optimizer.zero_grad()

        # Retain the graph to avoid clearing it before backward pass
        tot.backward(retain_graph=True)

        optimizer.step()

        total_loss += tot.item()

        print(f"Total Loss: {total_loss}")
        print("-" * 40)

    # Disable anomaly detection after running the code
    torch.autograd.set_detect_anomaly(False)

    for name, param in model.named_parameters():
        if param.requires_grad:
            print(f"Parameter name: {name}")
            print(f"Weights: {param.data}")

    return model


# Test

In [17]:
P_pi_b = action_probs_top_n_epsilon(q_table, 1, 0.4)
pi_b = experiment_actions(200, env, P_pi_b)
P_pi_e = action_probs_top_n_epsilon(q_table, 2, 0.05)
pi_e = experiment_actions(200, env, P_pi_e)

In [18]:
model_200 = CustomizableFeatureNet(input_dim=2, hidden_dims=[16, 32], output_dim=1, dtype = torch.float64)

In [19]:
test_200 = SCOPE_straight(model_200, 0.9, 10000, pi_b, P_pi_b, P_pi_e, dtype = torch.float64)

In [23]:
model_200 = train_var_scope(model_200, 1000, 0.0005, test_200)

Streaming output truncated to the last 5000 lines.
IS variance:  tensor(4.4229e-05, dtype=torch.float64)
SCOPE Var loss:  tensor(0.0008, dtype=torch.float64, grad_fn=<VarBackward0>)
Total Loss: 0.0008492256656322879
----------------------------------------
Epoch 22
IS variance:  tensor(4.4229e-05, dtype=torch.float64)
SCOPE Var loss:  tensor(0.0008, dtype=torch.float64, grad_fn=<VarBackward0>)
Total Loss: 0.000832036997470311
----------------------------------------
Epoch 23
IS variance:  tensor(4.4229e-05, dtype=torch.float64)
SCOPE Var loss:  tensor(0.0008, dtype=torch.float64, grad_fn=<VarBackward0>)
Total Loss: 0.0008178091443168292
----------------------------------------
Epoch 24
IS variance:  tensor(4.4229e-05, dtype=torch.float64)
SCOPE Var loss:  tensor(0.0008, dtype=torch.float64, grad_fn=<VarBackward0>)
Total Loss: 0.0008067588044517235
----------------------------------------
Epoch 25
IS variance:  tensor(4.4229e-05, dtype=torch.float64)
SCOPE Var loss:  tensor(0.0008, dtyp

# Test 200 0.99

In [35]:
P_pi_b = action_probs_top_n_epsilon(q_table, 1, 0.4)
pi_b = experiment_actions(200, env, P_pi_b)
P_pi_e = action_probs_top_n_epsilon(q_table, 2, 0.05)
# pi_e = experiment_actions(200, env, P_pi_e)

In [51]:
P_pi_b = action_probs_top_n_epsilon(q_table, 1, 0.4)
pi_b = experiment_actions(200, env_30, P_pi_b)
P_pi_e = action_probs_top_n_epsilon(q_table, 2, 0.05)
# pi_e = experiment_actions(200, env, P_pi_e)
model_200_0p99 = CustomizableFeatureNet(input_dim=2, hidden_dims=[6, 6], output_dim=1, dtype = torch.float64)
test_200_0p99 = SCOPE_straight(model_200_0p99, 0.99, 10000, pi_b, P_pi_b, P_pi_e, dtype = torch.float64)


Streaming output truncated to the last 5000 lines.
IS variance:  tensor(6.4534e-06, dtype=torch.float64)
SCOPE Var loss:  tensor(0.8547, dtype=torch.float64, grad_fn=<VarBackward0>)
Total Loss: 0.8546716723749704
----------------------------------------
Epoch 7
IS variance:  tensor(6.4534e-06, dtype=torch.float64)
SCOPE Var loss:  tensor(0.8345, dtype=torch.float64, grad_fn=<VarBackward0>)
Total Loss: 0.8344950296004069
----------------------------------------
Epoch 8
IS variance:  tensor(6.4534e-06, dtype=torch.float64)
SCOPE Var loss:  tensor(0.8157, dtype=torch.float64, grad_fn=<VarBackward0>)
Total Loss: 0.815702575610253
----------------------------------------
Epoch 9
IS variance:  tensor(6.4534e-06, dtype=torch.float64)
SCOPE Var loss:  tensor(0.7972, dtype=torch.float64, grad_fn=<VarBackward0>)
Total Loss: 0.7971985257757953
----------------------------------------
Epoch 10
IS variance:  tensor(6.4534e-06, dtype=torch.float64)
SCOPE Var loss:  tensor(0.7790, dtype=torch.float64

In [52]:
model_200_0p99 = train_var_scope(model_200_0p99, 200, 0.001, test_200_0p99)

Epoch 1
IS variance:  tensor(6.4534e-06, dtype=torch.float64)
SCOPE Var loss:  tensor(6.2321e-05, dtype=torch.float64, grad_fn=<VarBackward0>)
Total Loss: 6.232071548283213e-05
----------------------------------------
Epoch 2
IS variance:  tensor(6.4534e-06, dtype=torch.float64)
SCOPE Var loss:  tensor(5.8212e-05, dtype=torch.float64, grad_fn=<VarBackward0>)
Total Loss: 5.821238477719147e-05
----------------------------------------
Epoch 3
IS variance:  tensor(6.4534e-06, dtype=torch.float64)
SCOPE Var loss:  tensor(5.4394e-05, dtype=torch.float64, grad_fn=<VarBackward0>)
Total Loss: 5.439401945160914e-05
----------------------------------------
Epoch 4
IS variance:  tensor(6.4534e-06, dtype=torch.float64)
SCOPE Var loss:  tensor(5.0930e-05, dtype=torch.float64, grad_fn=<VarBackward0>)
Total Loss: 5.093024913095342e-05
----------------------------------------
Epoch 5
IS variance:  tensor(6.4534e-06, dtype=torch.float64)
SCOPE Var loss:  tensor(4.7785e-05, dtype=torch.float64, grad_fn=<

# Test 400 0.99

In [58]:
P_pi_b_400 = action_probs_top_n_epsilon(q_table, 1, 0.4)
pi_b_400 = experiment_actions(400, env, P_pi_b_400)
P_pi_e_400 = action_probs_top_n_epsilon(q_table, 2, 0.05)
# pi_e_400 = experiment_actions(400, env, P_pi_e_400)
model_400_0p99 = CustomizableFeatureNet(input_dim=2, hidden_dims=[6, 6], output_dim=1, dtype = torch.float64)
test_400_0p99 = SCOPE_straight(model_400_0p99, 0.99, 1000, pi_b_400, P_pi_b_400, P_pi_e_400, dtype = torch.float64)
model_400_0p99 = train_var_scope(model_400_0p99, 5, 0.001, test_400_0p99)

Epoch 1
IS variance:  tensor(0.0002, dtype=torch.float64)
SCOPE Var loss:  tensor(0.0864, dtype=torch.float64, grad_fn=<VarBackward0>)
Total Loss: 0.08636927251674584
----------------------------------------
Epoch 2
IS variance:  tensor(0.0002, dtype=torch.float64)
SCOPE Var loss:  tensor(0.2360, dtype=torch.float64, grad_fn=<VarBackward0>)
Total Loss: 0.23601228060459742
----------------------------------------
Epoch 3
IS variance:  tensor(0.0002, dtype=torch.float64)
SCOPE Var loss:  tensor(0.2284, dtype=torch.float64, grad_fn=<VarBackward0>)
Total Loss: 0.22842130839536168
----------------------------------------
Epoch 4
IS variance:  tensor(0.0002, dtype=torch.float64)
SCOPE Var loss:  tensor(0.2203, dtype=torch.float64, grad_fn=<VarBackward0>)
Total Loss: 0.22034274292801542
----------------------------------------
Epoch 5
IS variance:  tensor(0.0002, dtype=torch.float64)
SCOPE Var loss:  tensor(0.2122, dtype=torch.float64, grad_fn=<VarBackward0>)
Total Loss: 0.21219201524204384
-

# Test 600 0.99

In [48]:
P_pi_b_600 = action_probs_top_n_epsilon(q_table, 1, 0.4)
pi_b_600 = experiment_actions(600, env_30, P_pi_b_600)
P_pi_e_600 = action_probs_top_n_epsilon(q_table, 2, 0.05)
# pi_e_600 = experiment_actions(600, env, P_pi_e_600)
model_600_0p99 = CustomizableFeatureNet(input_dim=2, hidden_dims=[6, 6], output_dim=1, dtype = torch.float32)
test_600_0p99 = SCOPE_straight(model_600_0p99, 0.99, 10000, pi_b_600, P_pi_b_600, P_pi_e_600, dtype = torch.float32)
model_600_0p99 = train_var_scope(model_600_0p99, 5, 0.001, test_600_0p99)

Epoch 1
IS variance:  tensor(0.0007)
SCOPE Var loss:  tensor(0.0505, grad_fn=<VarBackward0>)
Total Loss: 0.05046245828270912
----------------------------------------
Epoch 2
IS variance:  tensor(0.0007)
SCOPE Var loss:  tensor(0.0493, grad_fn=<VarBackward0>)
Total Loss: 0.0493154413998127
----------------------------------------
Epoch 3
IS variance:  tensor(0.0007)
SCOPE Var loss:  tensor(0.0481, grad_fn=<VarBackward0>)
Total Loss: 0.04812745749950409
----------------------------------------
Epoch 4
IS variance:  tensor(0.0007)
SCOPE Var loss:  tensor(0.0469, grad_fn=<VarBackward0>)
Total Loss: 0.046937569975852966
----------------------------------------
Epoch 5
IS variance:  tensor(0.0007)
SCOPE Var loss:  tensor(0.0458, grad_fn=<VarBackward0>)
Total Loss: 0.04576045647263527
----------------------------------------
Parameter name: hidden_layers.0.weight
Weights: tensor([[ 0.1204,  0.1797],
        [ 0.1732,  0.2432],
        [ 0.6242,  0.4461],
        [ 0.4436,  0.0618],
        [ 

# Test 800 0.99

In [47]:
P_pi_b_800 = action_probs_top_n_epsilon(q_table, 1, 0.4)
pi_b_800 = experiment_actions(800, env_30, P_pi_b_800)
P_pi_e_800 = action_probs_top_n_epsilon(q_table, 2, 0.05)
# pi_e_800 = experiment_actions(800, env_30, P_pi_e_800)
model_800_0p99 = CustomizableFeatureNet(input_dim=2, hidden_dims=[6, 6], output_dim=1, dtype = torch.float32)
test_800_0p99 = SCOPE_straight(model_800_0p99, 0.99, 10000, pi_b_800, P_pi_b_800, P_pi_e_800, dtype = torch.float32)
model_800_0p99 = train_var_scope(model_800_0p99, 5, 0.001, test_800_0p99)

Epoch 1
IS variance:  tensor(6.2439e-07)
SCOPE Var loss:  tensor(0.1004, grad_fn=<VarBackward0>)
Total Loss: 0.10040785372257233
----------------------------------------
Epoch 2
IS variance:  tensor(6.2439e-07)
SCOPE Var loss:  tensor(0.1327, grad_fn=<VarBackward0>)
Total Loss: 0.13271035254001617
----------------------------------------
Epoch 3
IS variance:  tensor(6.2439e-07)
SCOPE Var loss:  tensor(0.1297, grad_fn=<VarBackward0>)
Total Loss: 0.12969112396240234
----------------------------------------
Epoch 4
IS variance:  tensor(6.2439e-07)
SCOPE Var loss:  tensor(0.1267, grad_fn=<VarBackward0>)
Total Loss: 0.12670785188674927
----------------------------------------
Epoch 5
IS variance:  tensor(6.2439e-07)
SCOPE Var loss:  tensor(0.1238, grad_fn=<VarBackward0>)
Total Loss: 0.12376594543457031
----------------------------------------
Parameter name: hidden_layers.0.weight
Weights: tensor([[ 0.1256,  0.1763],
        [ 0.1731,  0.2431],
        [ 0.6243,  0.4461],
        [ 0.4445, 

# Test 1000 0.99

In [45]:
P_pi_b_1000 = action_probs_top_n_epsilon(q_table, 1, 0.4)
pi_b_1000 = experiment_actions(1000, env_30, P_pi_b_1000)
P_pi_e_1000 = action_probs_top_n_epsilon(q_table, 2, 0.05)
# pi_e_1000 = experiment_actions(1000, env, P_pi_e_1000)
model_1000_0p99 = CustomizableFeatureNet(input_dim=2, hidden_dims=[6, 6], output_dim=1, dtype = torch.float32)
test_1000_0p99 = SCOPE_straight(model_1000_0p99, 0.90, 10000, pi_b_1000, P_pi_b_1000, P_pi_e_1000, dtype = torch.float32)
model_1000_0p99 = train_var_scope(model_1000_0p99, 5, 0.001, test_1000_0p99)

Epoch 1
IS variance:  tensor(1.3628e-08)
SCOPE Var loss:  tensor(0.0071, grad_fn=<VarBackward0>)
Total Loss: 0.0071372101083397865
----------------------------------------
Epoch 2
IS variance:  tensor(1.3628e-08)
SCOPE Var loss:  tensor(0.0059, grad_fn=<VarBackward0>)
Total Loss: 0.005885153077542782
----------------------------------------
Epoch 3
IS variance:  tensor(1.3628e-08)
SCOPE Var loss:  tensor(0.0057, grad_fn=<VarBackward0>)
Total Loss: 0.005744975060224533
----------------------------------------
Epoch 4
IS variance:  tensor(1.3628e-08)
SCOPE Var loss:  tensor(0.0056, grad_fn=<VarBackward0>)
Total Loss: 0.005605767946690321
----------------------------------------
Epoch 5
IS variance:  tensor(1.3628e-08)
SCOPE Var loss:  tensor(0.0055, grad_fn=<VarBackward0>)
Total Loss: 0.005469260271638632
----------------------------------------
Parameter name: hidden_layers.0.weight
Weights: tensor([[-0.3398,  0.4284],
        [ 0.6470, -0.5601],
        [-0.0836, -0.5419],
        [ 0.